# Map

## Importing data and creating new column to use it to get geografical location

In [1]:
import pandas as pd
df = pd.read_csv('DataGruppo2.csv', encoding='windows-1252')
df['City_full'] = df[['City','State','Country',]].agg(', '.join, axis=1)


In [2]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,City_full
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,"Henderson, Kentucky, United States"
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,"Henderson, Kentucky, United States"
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,"Los Angeles, California, United States"
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,"Fort Lauderdale, Florida, United States"
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,"Fort Lauderdale, Florida, United States"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

## Create a separate excel file for easier work

In [4]:
df_map = df.groupby('City_full')['Sales'].sum()
df_map = df_map.to_frame()

df_map = df_map.reset_index()
df_map.to_excel('Map.xlsx')

df_map

,City_full,Sales
0,"Aberdeen, South Dakota, United States",25.500
1,"Abilene, Texas, United States",1.392
2,"Akron, Ohio, United States",2729.986
3,"Albuquerque, New Mexico, United States",2220.160
4,"Alexandria, Virginia, United States",5519.570
...,...,...
599,"Woonsocket, Rhode Island, United States",195.550
600,"Yonkers, New York, United States",7657.666
601,"York, Pennsylvania, United States",817.978
602,"Yucaipa, California, United States",50.800


## Bubble Map

In [5]:
import folium
import pandas as pd
from geopy.geocoders import Nominatim

df = pd.read_excel('Map.xlsx')

# Create a map, start on the United States
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Loop through the cities and add a circle marker for each one
for index, row in df.iterrows():
    city = row['City_full']
    sales = row['Sales']
    location = None
    
    # Use Geopy to find the latitude and longitude of the city
    geolocator = Nominatim(user_agent="Jeve_map")
    location = geolocator.geocode(city)
    
    # If the location was found, add a circle marker to the map
    if location is not None:
        folium.CircleMarker(location=[location.latitude, location.longitude], radius=sales/1000, color='blue', fill=True, fill_color='blue', fill_opacity=0.3, tooltip=f"{city}: ${sales}K").add_to(m)

# Display the map
m

## Heat Map

In [14]:
import folium
import pandas as pd
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap

In [15]:
df = pd.read_excel('Heat_map.xlsx')
# Create a map, start on the United States

m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Create a dictionary with city names and sales amounts
sales_dict = dict(zip(df['City_full'], df['Sales_sum']))
sales_dict1 = dict(zip(df['City_full'], df['Profit_sum']))
sales_dict2 = dict(zip(df['City_full'], df['Sales_mean']))
sales_dict3 = dict(zip(df['City_full'], df['Profit_mean']))

In [16]:
# Use Geopy to find the latitude and longitude of each city
geolocator = Nominatim(user_agent="Jeve_heat_maper")
locations = {city: geolocator.geocode(city) for city in sales_dict.keys()}

# Create a list of latitudes and longitudes
latitudes = [location.latitude for location in locations.values()]
longitudes = [location.longitude for location in locations.values()]

In [17]:
# Create a heatmap layer with the needed parameter
heatmap_layer = folium.FeatureGroup(name='Sales sum')
heatmap_layer.add_child(HeatMap(list(zip(latitudes, longitudes, list(sales_dict.values()))), name='Sales_sum', min_opacity=0.2, radius=20, blur=15))
heatmap_layer1 = folium.FeatureGroup(name='Profit sum')
heatmap_layer1.add_child(HeatMap(list(zip(latitudes, longitudes, list(sales_dict1.values()))), name='Profit_sum', min_opacity=0.2, radius=20, blur=15))
heatmap_layer2 = folium.FeatureGroup(name='Sales mean')
heatmap_layer2.add_child(HeatMap(list(zip(latitudes, longitudes, list(sales_dict2.values()))), name='Sales_mean', min_opacity=0.2, radius=20, blur=15))
heatmap_layer3 = folium.FeatureGroup(name='Profit mean')
heatmap_layer3.add_child(HeatMap(list(zip(latitudes, longitudes, list(sales_dict3.values()))), name='Profit_mean', min_opacity=0.2, radius=20, blur=15))


# Add the heatmap layer to the map
m.add_child(heatmap_layer)
m.add_child(heatmap_layer1)
m.add_child(heatmap_layer2)
m.add_child(heatmap_layer3)

# Display the map
folium.LayerControl().add_to(m)
m